<center> <b>Machine Learning - SBU FALL 2024</b></center> 

In [ ]:
student_number = ''
Name = ''
Last_Name = ''

In this notebook, you are expected to implement a fully functional MLP (Multi-Layer Perceptron) neural network from scratch. 
You are not allowed to use any libraries (including numpy). You will use the **Iris dataset** for training and testing your network, focusing on reducing the error on this dataset. 


**modify iris dataset to a version compatible with this task : binary classifiaction of if a flower is setosa (1) or not(-1)**

# Automatic Differentiation

Automatic differentiation has two main methods: forward mode and reverse mode. 
PyTorch uses the reverse mode approach, and we will also use this method in this project.

To learn this concept, simply click on this [link](https://auto-ed.readthedocs.io/en/latest/mod3.html#i-the-basics-of-reverse-mode) 
and read only the section "Intuition for Example An IV" up to the end of step six.

Essentially, you need to consider a data structure to build a computational graph. 
By calling the `backward` function on the network's output, you can compute the derivative of the output 
with respect to all weights and biases of the network. (In this case, our network has only one output.)


## Visualization Tool

In [ ]:
from graphviz import Digraph
import random
import math
from math import exp

def trace(root):
    nodes, edges = set(), set()

    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v.children:
                edges.add((child, v))
                build(child)

    build(root)
    return nodes, edges


def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir})  # , node_attr={'rankdir': 'TB'})

    for n in nodes:
        dot.node(name=str(id(n)), label="{ %s | data %.4f | grad %.4f }" % (n.label, n.value, n.grad), shape='record')
        if n.operator:
            dot.node(name=str(id(n)) + n.operator, label=n.operator)
            dot.edge(str(id(n)) + n.operator, str(id(n)))

    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2.operator)

    return dot

## Tensor

In [ ]:
class Tensor:

  def __init__(self, value, label='', children=(), operator=None):
    self.value = value
    self.children = set(children)
    self.operator = operator
    self.grad = 0
    self._backward = lambda  : None
    self.label = label


  def __repr__(self) -> str:
    return f"Tensor(label = {self.label}, value = {self.value}, grad = {self.grad}, operator = {self.operator})"

  # normal add
  def __add__(self, other):

    other = other if isinstance(other, Tensor) else Tensor(other)

    out_value = self.value + other.value
    out = Tensor(out_value, children=(self, other), operator='+')

    def backward():
      self.grad = 1 * out.grad
      other.grad = 1 * out.grad

    out._backward = backward

    return out

  # reverse add
  def __radd__(self, other):
    return self + other

  def __sub__(self, other):
    # TODO

  def __mul__(self, other):
    # TODO

  def __rmul__(self, other):
    # TODO

  def __truediv__(self, other):
    # TODO

  def __pow__(self, other):
    # TODO

  def backward(self):
    # TODO

# Activation Functions

In [ ]:
class F:

  @staticmethod
  def tanh(x: Tensor) -> Tensor:
    # TODO

    def backward():
    # TODO

# Neuron, Layer & MLP (Forward Section)

In [ ]:
class Neuron:

  def __init__(self, input_size):

    self.weights = [Tensor(random.uniform(-1, 1)) for i in range(input_size)]
    self.bias = Tensor(random.uniform(-1, 1))

  def forward(self, x):
    # TODO

  def __call__(self, x):
    # TODO

  def parameters(self):
    # TODO

  def __print__(self):
    # TODO

In [ ]:
class Layer:

  def __init__(self, input_size, output_size):
    self.neurons = [Neuron(input_size) for _ in range(output_size)]

  def forward(self, x):
    # TODO

  def __call__(self, x):
    # TODO

  def parameters(self):
    # TODO

  def __print__(self):
    # TODO

# The MLP Class Structure

The `MLP` class is expected to have three main methods, which you should implement with the same structure:

1. **`forward` Method:**  
   This method performs calculations on the input and returns the output.

2. **`__call__` Method:**  
   This method simply calls the `forward` method. Essentially, we want to pass the input to the model in this way: `model(x)`.

3. **`parameters` Method:**  
   This method returns all the weights of the network in a list.

---

# Layers in the MLP

The `MLP` class itself consists of several layers (referred to as `Layer`), which are the actual layers of the neural network. Each layer needs to know:
- The dimensions of the input data.
- The dimensions of the output data.

In this project, all inputs are vectors. Each `Layer` consists of several neurons. For example:
- If a layer receives a 7-dimensional vector as input (input size) and produces a 4-dimensional vector as output, then:
  - The layer should have 4 neurons.
  - Each neuron should have 7 weights and 1 bias.

You should implement the details and structure of the layers and neurons according to the given explanation.


In [ ]:
class MLP:

  def __init__(self, input_size, layer_sizes):
    layers_total = [input_size] + layer_sizes
    self.layers = [Layer(layers_total[i], layers_total[i+1]) for i in range(len(layer_sizes))]

  def forward(self, x):
    # TODO

  def __call__(self, x):
    # TODO

  def parameters(self):
    # TODO

  def __print__(self):
    # TODO

# Optimizer

# Optimizer

Similar to the first project, the `optimizer` should have access to the network's weights. 
This time, it must update them based on their derivatives and the value of the `lr` parameter (learning rate).

- **`step` Method:**  
  This method functions similarly to the `update` method in the previous project. It updates the weights of the network.

- **`grad_zero` Method:**  
  An additional method called `grad_zero` is included, whose functionality has been explained. It is used to reset the gradients of the weights to zero after each update step.


In [ ]:
class Optimizer:
  def __init__(self, parameters, lr):
    self.parameters = parameters
    self.lr = lr

  def zero_grad(self):
    # TODO

  def step(self):
    # TODO

# Training Part

Prepeare the dataset in this section in bellow code snippet we place a toy example dataset just for better clarification

In [ ]:
X = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]

Y = [1.0, -1.0, -1.0, 1.0]

In [ ]:
# TODO

## Loss Function (SE)

In [ ]:
from typing import List

def criterion(y_hats: List[Tensor], Y) -> Tensor:
  return sum([(y_hat - y)**2 for y_hat, y in zip(y_hats, Y)])

## Training Loop

# Training Steps for the Model

1. **Calculate Model Predictions:**  
   For each input `x`, compute the output of the model, `y_hat`.

2. **Compute Error:**  
   Calculate the error of the predictions using the Mean Squared Error (MSE) loss function for simplicity.

3. **Reset Gradients of Network Variables:**  
   Set the gradients of all variables in the network to zero. Once you implement automatic differentiation, you will understand why this step is necessary.

4. **Compute Derivatives:**  
   This is the most challenging part of the project. When this function is called, you need to calculate the derivative of the `loss` with respect to all weights and biases of the network.  
   To implement this, you will use **Automatic Differentiation (AutoDiff)**.

5. **Update Network Weights:**  
   The `optimizer` will use the derivatives of the `loss` with respect to all weights and biases to update them in a direction that reduces the error in the next step.


In [ ]:
n_epochs = 20

input_size = 3 # Number of features in the input layer
layer_sizes = [2, 3, 1] # Number of neurons in each hidden and output layer
model = MLP(input_size, layer_sizes)

for _ in range(n_epochs):
  # Forward pass: Compute predictions for the entire dataset
  y_hats = ...

  # Compute the loss
  loss = ...

  # Zero the gradients to prevent accumulation from previous iterations
  optim.zero_grad()

  # Backward pass: Compute the gradient of the loss function with respect to model parameters
  loss.backward()

  # Update the model parameters using the optimizer
  optim.step()

draw_dot(loss)